# side model

In [1]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_90m.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(29741, 2960)
(29741, 7)


5h_dft_dom_cycle_lag39           293
5h_dft_dom_cycle_lag38           290
5h_dft_dom_cycle_lag37           286
5h_dft_dom_cycle_lag36           283
5h_dft_dom_cycle_lag35           280
                                ... 
90m_td_sequential_buy_stealth      0
90m_adaptive_rsi                   0
90m_fisher                         0
90m_comb_spectrum_pwr_26           0
90m_adaptive_stochastic            0
Length: 2960, dtype: int64

In [2]:
df_features = df_features[293:]
label_side = label_side[293:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(29448, 2960)
(29448, 7)


25m_ac_16                  0
5h_adaptive_bp_dt_lag6     0
5h_adaptive_bp_dt_lag1     0
5h_adaptive_bp_dt_lag12    0
5h_adaptive_bp_dt_lag13    0
                          ..
90m_adaptive_bp_lag1       0
90m_adaptive_bp_lag2       0
90m_adaptive_bp_lag3       0
90m_adaptive_bp_lag34      0
5h_williams_r_lag9         0
Length: 2960, dtype: int64

In [3]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(29448, 2960)
(array([0, 1]), array([13947, 15501]))
(array([0, 1]), array([2841, 3049]))


In [6]:
import optuna
import lightgbm as lgb
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

[I 2025-03-25 14:23:21,392] A new study created in memory with name: no-name-c975be3a-6280-484d-a42c-5aed97cbd0eb


[I 2025-03-25 14:23:54,379] Trial 0 finished with value: 0.854219980145942 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 294, 'max_depth': 726, 'min_gain_to_split': 0.7310472583785544, 'min_data_in_leaf': 394, 'lambda_l1': 49.798172368777784, 'lambda_l2': 5.105083995104038, 'num_boost_round': 1839}. Best is trial 0 with value: 0.854219980145942.


precision: 0.8023469013568023, recall: 0.7176123319121023, f1: 0.7576177285318559


[I 2025-03-25 14:24:47,754] Trial 1 finished with value: 0.8448774440792182 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 148, 'max_depth': 128, 'min_gain_to_split': 0.9310915944614735, 'min_data_in_leaf': 72, 'lambda_l1': 69.83756599782045, 'lambda_l2': 56.1211257655214, 'num_boost_round': 1986}. Best is trial 0 with value: 0.854219980145942.


precision: 0.7917578409919767, recall: 0.7120367333551985, f1: 0.7497841478155759


...

precision: 0.8056460369163952, recall: 0.730075434568711, f1: 0.7660013764624914


[I 2025-03-25 14:47:32,930] Trial 98 finished with value: 0.8574514884136366 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 229, 'max_depth': 382, 'min_gain_to_split': 0.20847462004452544, 'min_data_in_leaf': 336, 'lambda_l1': 42.23862257728599, 'lambda_l2': 5.5421568137612685, 'num_boost_round': 430}. Best is trial 96 with value: 0.8593181023454872.


precision: 0.796392646548734, recall: 0.7530337815677272, f1: 0.7741065407956844


[I 2025-03-25 14:47:48,656] Trial 99 finished with value: 0.8532472490562164 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 233, 'max_depth': 412, 'min_gain_to_split': 0.2568420786891065, 'min_data_in_leaf': 342, 'lambda_l1': 29.25271378994739, 'lambda_l2': 4.987475262471049, 'num_boost_round': 508}. Best is trial 96 with value: 0.8593181023454872.


precision: 0.7990687679083095, recall: 0.7317153164972122, f1: 0.7639102893340182


In [7]:
study.best_params

{'is_unbalance': False,
 'extra_trees': True,
 'boosting': 'dart',
 'num_leaves': 246,
 'max_depth': 336,
 'min_gain_to_split': 0.21627522249400022,
 'min_data_in_leaf': 373,
 'lambda_l1': 29.425167496902432,
 'lambda_l2': 4.788852157486146,
 'num_boost_round': 321}

In [8]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [9]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [10]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.8593181023454872


In [12]:
model.save_model("model/model_side.txt")
prod_model.save_model("model/model_side_prod.txt")

# meta model

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_90m.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

5h_dft_dom_cycle_lag39              293
5h_dft_dom_cycle_lag38              290
5h_dft_dom_cycle_lag37              286
5h_dft_dom_cycle_lag36              283
5h_dft_dom_cycle_lag35              280
                                   ... 
90m_mod_stochastic                    0
90m_mod_rsi                           0
90m_td_sequential_buy_aggressive      0
90m_conv_38                           0
side_model_res                        0
Length: 2960, dtype: int64

In [3]:
df_features = df_features[293:]
meta_label = meta_label[293:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(29448, 2960)
(29448, 7)


25m_acc_swing_index_ddt_lag14      0
5h_adaptive_rsi_dt_lag14           0
5h_adaptive_rsi_ddt_lag23          0
5h_adaptive_rsi_ddt_lag24          0
5h_adaptive_rsi_ddt_lag25          0
                                  ..
90m_adaptive_stochastic_dt_lag9    0
90m_adaptive_stochastic_lag10      0
90m_adaptive_stochastic_lag11      0
90m_adaptive_stochastic_lag12      0
side_model_res                     0
Length: 2960, dtype: int64

In [4]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(29448, 2960)
(array([0, 1]), array([ 4031, 19527]))
(array([0, 1]), array([1324, 4566]))


In [5]:
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-25 15:25:22,935] A new study created in memory with name: no-name-b91ea805-f5ad-4394-9796-775d0c56dd28
[I 2025-03-25 15:25:30,579] Trial 0 finished with value: 0.6974843344894767 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 37, 'max_depth': 458, 'min_gain_to_split': 0.1570053000921059, 'min_data_in_leaf': 230, 'lambda_l1': 40.23842661740964, 'lambda_l2': 41.125539338415564, 'num_boost_round': 399}. Best is trial 0 with value: 0.6974843344894767.


precision: 0.8562351072279587, recall: 0.7082785808147175, f1: 0.7752606975907946, auc: 0.7194246056164505, fbeta: 0.6974843344894767


[I 2025-03-25 15:25:46,342] Trial 1 finished with value: 0.6941060530522182 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 79, 'max_depth': 501, 'min_gain_to_split': 0.506281077184619, 'min_data_in_leaf': 223, 'lambda_l1': 64.13722050997315, 'lambda_l2': 63.47778634733558, 'num_boost_round': 1347}. Best is trial 0 with value: 0.6974843344894767.


precision: 0.8550146315509444, recall: 0.703898379325449, f1: 0.7721321321321322, auc: 0.7168887865518552, fbeta: 0.6941060530522182


[I 2025-03-25 15:25:59,165] Trial 2 finished with value: 0.5200563560004148 and parameters: {'extra_trees': True, 'boosting': 'dart', 'num_leaves': 315, 'max_depth': 643, 'min_gain_to_split': 0.6716778047425648, 'min_data_in_leaf': 40, 'lambda_l1': 54.03934923542029, 'lambda_l2': 62.719923545570595, 'num_boost_round': 636}. Best is trial 0 with value: 0.6974843344894767.


...

precision: 0.8075502667213788, recall: 0.8620236530880421, f1: 0.8338983050847457, auc: 0.7030076170512908, fbeta: 0.7227154236662015


[I 2025-03-25 16:02:42,657] Trial 98 finished with value: 0.7078639898479256 and parameters: {'extra_trees': False, 'boosting': 'dart', 'num_leaves': 266, 'max_depth': 521, 'min_gain_to_split': 0.5294607712310383, 'min_data_in_leaf': 116, 'lambda_l1': 46.59421215660101, 'lambda_l2': 37.89823297364665, 'num_boost_round': 407}. Best is trial 9 with value: 0.7281884610273439.


precision: 0.8442622950819673, recall: 0.7444152431011827, f1: 0.7912011173184358, auc: 0.7144957210327747, fbeta: 0.7078639898479256


[I 2025-03-25 16:02:58,238] Trial 99 finished with value: 0.7238374041412796 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 252, 'max_depth': 208, 'min_gain_to_split': 0.3833943539995619, 'min_data_in_leaf': 101, 'lambda_l1': 2.0378999886332636, 'lambda_l2': 23.57321806052342, 'num_boost_round': 788}. Best is trial 9 with value: 0.7281884610273439.


precision: 0.8088962108731467, recall: 0.8602715724923347, f1: 0.8337932498407982, auc: 0.7021424941740674, fbeta: 0.7238374041412796


In [7]:
study.best_params

{'extra_trees': False,
 'boosting': 'gbdt',
 'num_leaves': 328,
 'max_depth': 720,
 'min_gain_to_split': 0.41060469559382395,
 'min_data_in_leaf': 123,
 'lambda_l1': 23.99081793714736,
 'lambda_l2': 28.909517535016796,
 'num_boost_round': 573}

In [8]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [9]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [11]:
model.save_model("model/model_meta.txt")
prod_model.save_model("model/model_meta_prod.txt")